In [1]:
import requests
import pandas as pd
import json

In [2]:
def run_sql(sql_query, table, fields=False):
    sql_query = sql_query.replace('TABLE_PLACEHOLDER',table)
    source = requests.get(f'https://data.ibb.gov.tr/api/3/action/datastore_search_sql?sql={sql_query}')
    try:
        result = pd.DataFrame(json.loads(source.text)['result']['records'])
        if fields:
            return pd.DataFrame(json.loads(source.text)['result']['fields'])
    except KeyError:
        result = json.loads(source.text)
    return result

In [3]:
run_sql('SELECT * FROM "TABLE_PLACEHOLDER" LIMIT 10', "db9c7fb3-e7f9-435a-92f4-1b917e357821", True)

,type,id
0,int4,_id
1,tsvector,_full_text
2,text,DATE_TIME
3,text,LONGITUDE
4,text,LATITUDE
5,text,GEOHASH
6,text,MINIMUM_SPEED
7,text,MAXIMUM_SPEED
8,text,AVERAGE_SPEED
9,text,NUMBER_OF_VEHICLES


In [7]:
columns = run_sql('SELECT * FROM "TABLE_PLACEHOLDER" LIMIT 10', "db9c7fb3-e7f9-435a-92f4-1b917e357821", True)['id'].tolist()

In [14]:
run_sql('SELECT * FROM "TABLE_PLACEHOLDER" LIMIT 10', "db9c7fb3-e7f9-435a-92f4-1b917e357821")

,DATE_TIME,NUMBER_OF_VEHICLES,MINIMUM_SPEED,AVERAGE_SPEED,_full_text,LONGITUDE,GEOHASH,MAXIMUM_SPEED,LATITUDE,_id
0,2020-01-03 13:00:00,105,9,53,"'-01':2 '-03':3 '00':5,6 '105':13 '13':4 '2020...",28.8446044921875,sxk9bk,78,41.1135864257812,1
1,2020-01-16 07:00:00,68,6,27,"'-01':2 '-16':3 '00':5,6 '07':4 '2020':1 '27':...",29.1522216796875,sxk9r3,64,41.0092163085938,2
2,2020-01-17 20:00:00,11,53,72,"'-01':2 '-17':3 '00':5,6 '11':13 '20':4 '2020'...",28.4710693359375,sxk1rz,96,41.0421752929688,3
3,2020-01-14 22:00:00,216,17,73,"'-01':2 '-14':3 '00':5,6 '128':11 '17':10 '202...",29.3609619140625,sxkbhn,128,40.8169555664062,4
4,2020-01-17 20:00:00,123,16,55,"'-01':2 '-17':3 '00':5,6 '123':13 '16':10 '20'...",28.8446044921875,sxk906,82,40.9707641601562,5
5,2020-01-20 10:00:00,88,68,93,"'-01':2 '-20':3 '00':5,6 '10':4 '152':11 '2020...",29.2950439453125,sxkcfe,152,41.1080932617188,6
6,2020-01-24 16:00:00,299,6,49,"'-01':2 '-24':3 '00':5,6 '16':4 '2020':1 '28.7...",28.7127685546875,sxk3jq,91,40.9927368164062,7
7,2020-01-27 13:00:00,167,6,15,"'-01':2 '-27':3 '00':5,6 '13':4 '15':12 '167':...",28.8995361328125,sxk99s,46,41.0696411132812,8
8,2020-01-06 20:00:00,287,6,59,"'-01':2 '-06':3 '00':5,6 '114':11 '20':4 '2020...",28.8226318359375,sxk3pz,114,40.9982299804688,9
9,2020-01-24 17:00:00,277,6,33,"'-01':2 '-24':3 '00':5,6 '17':4 '2020':1 '277'...",28.9764404296875,sxk9g3,77,41.0971069335938,10


In [29]:
col_stats = ''
for col in columns[6:]:
    col_stats+= f'MIN("{col}"::integer) AS MIN_{col},MAX("{col}"::integer) AS MAX_{col},AVG("{col}"::integer) AS AVG_{col},'    
col_stats = col_stats[:-1]

In [31]:
sql_query = f"""
SELECT {col_stats}
FROM "TABLE_PLACEHOLDER"
LIMIT 10
"""
run_sql(sql_query, "db9c7fb3-e7f9-435a-92f4-1b917e357821").T

,0
avg_minimum_speed,21.3725877298618721
max_number_of_vehicles,1216
min_average_speed,8
min_number_of_vehicles,11
max_minimum_speed,103
min_minimum_speed,0
avg_number_of_vehicles,96.2364468803815075
avg_maximum_speed,100.0673870109225785
avg_average_speed,55.1073042486060767
min_maximum_speed,14


In [40]:
col_numeric = ''
for col in columns[6:]:
    col_numeric+= f'"{col}"::integer,'    
col_numeric = col_numeric[:-1]
col_numeric

'"MINIMUM_SPEED"::integer,"MAXIMUM_SPEED"::integer,"AVERAGE_SPEED"::integer,"NUMBER_OF_VEHICLES"::integer'

In [63]:
daily_avgs = ''
for col in columns[6:]:
    if col.startswith('NUM'):
        daily_avgs += f'MIN("{col}") AS MIN_{col},'
        daily_avgs += f'MAX("{col}") AS MAX_{col},'
        daily_avgs += f'AVG("{col}")::numeric(5,2) AS AVG_{col},'
    else:
        if col[:3] == 'AVE':
            daily_avgs+= f'{col[:3].replace("E","G")}("{col}")::numeric(5,2) AS "{col}",'
        else:
            daily_avgs+= f'{col[:3].replace("E","G")}("{col}") AS "{col}",'
daily_avgs = daily_avgs[:-1]

In [65]:
sql_query = f'''
WITH temp_table AS (
SELECT DATE_TRUNC('day', "DATE_TIME"::date)::date AS "DATE", {col_numeric}
FROM "TABLE_PLACEHOLDER" 
)

SELECT "DATE", {daily_avgs}
FROM temp_table
GROUP BY "DATE"
ORDER BY "DATE"
'''
df = run_sql(sql_query, "db9c7fb3-e7f9-435a-92f4-1b917e357821")

In [66]:
df.to_csv('traffic_daily_agg.csv', sep=',', index=False)
df.to_csv('traffic_daily_agg.csv', sep=',', mode='a', header=False, index=False)

In [67]:
pd.read_csv('traffic_daily_agg.csv')

,max_number_of_vehicles,MINIMUM_SPEED,min_number_of_vehicles,AVERAGE_SPEED,avg_number_of_vehicles,MAXIMUM_SPEED,DATE
0,539,1,11,58.03,67.08,219,2020-01-01
1,1134,1,11,54.22,108.61,217,2020-01-02
2,1009,1,11,55.14,103.68,222,2020-01-03
3,1051,1,11,56.74,95.52,223,2020-01-04
4,540,1,11,57.57,69.30,210,2020-01-05
...,...,...,...,...,...,...,...
57,1137,1,11,55.66,113.43,214,2020-01-27
58,1195,1,11,54.56,116.55,205,2020-01-28
59,1169,1,11,54.32,109.15,232,2020-01-29
60,1187,1,11,54.67,113.14,230,2020-01-30


In [374]:
traffic_link = 'https://data.ibb.gov.tr/dataset/hourly-traffic-density-data-set'
weather_link = 'https://data.ibb.gov.tr/dataset/meteorology-observation-station-data-set'

In [375]:
from data_scrape import get_table_names

In [376]:
get_table_names(traffic_link)

['db9c7fb3-e7f9-435a-92f4-1b917e357821',
 '5fb30ee1-e079-4865-a8cd-16efe2be8352',
 'efff9df8-4f40-4a46-8c99-2b3b4c5e2b8c',
 '9ead7895-27fb-4aed-847f-ffe1504c36fa',
 '5c0da73a-2fd6-4f98-90fe-aa32ce98b607',
 '62099013-e557-4d23-a2c0-70f7ee89c3b9',
 'e5fb99b3-afa0-4a9d-9bc8-cf98940da082',
 'dc40309d-7fd6-43e2-ad85-5db9db133a5b',
 'ef34bd55-86d8-4459-a710-79de30a45be2',
 '949d4a3b-91d2-4c56-b82f-4ef081e39c45',
 '93f996f1-70da-4500-951a-693c7e7066f6',
 '3e3161d8-7668-4694-829c-9179b41a775b',
 'fb7094a3-cf2f-46a6-996a-f6a9c5f3b9be',
 '395811ac-4152-4e04-88ef-8d4e30e6ac17',
 'fdbc8e2f-0cf1-4952-b50f-df8f40d5a649',
 '1eb158e8-8da7-4572-9825-108714a8856e',
 '00d72836-d035-462d-a66e-408883216195']